# Supervised learning with Scikit-learn
[Scikit-learn](http://scikit-learn.org/) is a Python module that implements some well-known machine learning algorithms. We will use [Matplotlib](http://matplotlib.org/) for visualizing data.

In [105]:
import sklearn
import matplotlib.pyplot as plt
from ipywidgets import interactive, widgets
from IPython.display import display, Audio
print(f"You have Scikit-learn {sklearn.__version__} installed")

You have Scikit-learn 1.2.0 installed


We generate a slider to set sample frequency for the whole notebook

In [106]:
ws = {'description_width': 'initial'}
sample_rate = widgets.IntSlider(40000, min=10000, max=50000, description="Sample frequency (hZ)", style=ws)
display(sample_rate)

IntSlider(value=40000, description='Sample frequency (hZ)', max=50000, min=10000, style=SliderStyle(descriptio…

In [107]:
import numpy as np
import numpy.typing as npt
def generate_noisy_signal(
    sample_frequency: int,
    amplitudes: npt.NDArray[np.float64],
    frequencies: npt.NDArray[np.float64],
    phases: npt.NDArray[np.float64], noise: float=1, T:float=2.):
    N = int(T*sample_frequency)
    t = np.linspace(0, T, N, endpoint=False)
    assert len(frequencies) == len(phases)
    assert len(amplitudes) == len(phases)
    n = noise*np.random.randn(len(t))
    return t, n+sum([A*np.cos(2*np.pi*f*t+p) for (A,f,p) in zip(amplitudes, frequencies, phases)])

We generate some audio signals, defined by two frequencies, including noise.
Use the widget below to play the sounds corresponding to different input.

In [108]:
@widgets.interact(
    A=widgets.FloatSlider(0.5, min=0, max=4, description="Amplitude of wave 1", style=ws),
    B=widgets.FloatSlider(3, min=0, max=4, description="Amplitude of wave 2", style=ws),
    fA=widgets.IntSlider(100, min=20, max=20000, description="Frequency of wave 1", style=ws),
    fB=widgets.IntSlider(10000, min=20, max=20000, description="Frequency of wave 2", style=ws),
    noise=widgets.FloatSlider(3, min=0, max=4, description="Amplitude of noise", style=ws)
)
def generate_sound(A: float, B:float, fA:float, fB:float, noise: float):
    T = 2
    pA, pB = 0, 0
    print(sample_rate.value)
    _, signal = generate_noisy_signal(sample_rate.value, np.array([A, B]), np.array([fA, fB]), np.array([pA, pB]),
                                      noise=noise, T=T)
    audio = Audio(signal, rate=sample_rate.value, autoplay=True)
    display(audio)



interactive(children=(FloatSlider(value=0.5, description='Amplitude of wave 1', max=4.0, style=SliderStyle(des…

Next, we extract a few features:
1. The expected value `E` and variance `S` of the signal
2. The RMS values of the PSD over 10 windows.

In [109]:
def extract_features(signal:npt.NDArray[np.float64], num_windows:int=10):
    E = 1/len(signal) * sum(signal)
    S = 1/len(signal) * sum((signal-E)**2)
    rfft = np.fft.rfft(signal)
    f_per_band = len(rfft)//num_windows
    rem = len(rfft) % num_windows
    freq_per_band = np.full(num_windows, f_per_band, dtype=np.int32)
    freq_per_band[:rem]+=1
    offsets= np.zeros(num_windows+1, dtype=np.int32)
    offsets[1:] = np.cumsum(freq_per_band)
    RMS = np.zeros(num_windows, dtype=np.float64)
    for i in range(num_windows):
        amps = np.abs(rfft[offsets[i]:offsets[i+1]])**2
        RMS[i] = np.sqrt(1/len(amps)*np.sum(amps**2))
    fft = np.fft.fft(signal)
    mag = np.abs(rfft)
    return np.hstack([E, S, RMS])


We now need to generate a database of labeled signals, that we know are high-frequent. We define a high frequent sound as any frequency over 8 kHz. We label high-frequency signals with `2`, low frequency with `1`.

In [110]:
from numpy.random import MT19937, RandomState, SeedSequence
rs = RandomState(MT19937(SeedSequence(123456789)))

def generate_database(num_samples:int=100, T:float=2, num_windows:int=2):
    A = rs.uniform(-10, 10, size=num_samples)
    phase = np.pi * rs.random(num_samples)
    noise = rs.uniform(-2, 2, size=num_samples) * rs.random_integers(0,1, size=num_samples)
    frequencies = rs.uniform(0, 20000, size=num_samples) 
    features = np.zeros((num_samples, num_windows+2))
    labels = np.zeros(num_samples, dtype=np.int32)
    for i in range(num_samples):
        _, signal = generate_noisy_signal(sample_rate.value, A[i:i+1], frequencies[i:i+1],
                                          phase[i:i+1],noise[i:i+1], T)
        features[i] = extract_features(signal, num_windows)
        labels[i] = 1 + np.int32(frequencies[i] > 8e3)
    target_map = {1:"low", 2:"high"}
    feature_names = ["Expected value", "Variance"] + [f"RMS of PSD of {i+1}th window" for i in range(num_windows)]

    return features, labels, feature_names, target_map

num_samples = 250
num_windows = 10
features, labels, feature_names, target_map = generate_database(num_samples=num_samples, num_windows=num_windows)

/tmp/ipykernel_10039/1152432844.py:7: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  noise = rs.uniform(-2, 2, size=num_samples) * rs.random_integers(0,1, size=num_samples)


As the features are `num_windows + 2`-dimensional, we cannot visualize it easily. However, we can visualize 2 features at the time. We observe that it is not easy to spot clear correlation between each individual feature

In [113]:
%matplotlib inline
@widgets.interact(
    f0=widgets.IntSlider(0, 0, num_windows+1, description="First feature"),
    f1=widgets.IntSlider(1, 0, num_windows+1, description="Second feature")
)
def visualize_features(f0:int, f1:int):
    target_labels = list(target_map.keys())
    formatter = plt.FuncFormatter(lambda i, *args: target_map[i])

    plt.scatter(features[:,f0], features[:,f1], c=labels,
                cmap=plt.cm.get_cmap('viridis', len(target_labels)))
    plt.colorbar(ticks=target_labels, format=formatter)

    plt.xlabel(feature_names[f0])
    plt.ylabel(feature_names[f1])


interactive(children=(IntSlider(value=0, description='First feature', max=11), IntSlider(value=1, description=…

We use the K-nearest neighborhood to classify the following signal
$x(t) = 10 \cos(2\pi\cdot5000 t) + 5 \sin(2\pi\cdot 200 t)$

In [123]:
%matplotlib inline
from sklearn.neighbors import KNeighborsClassifier

@widgets.interact(
    A=widgets.FloatSlider(0.5, min=0, max=4, description="Amplitude of wave 1", style=ws),
    B=widgets.FloatSlider(3, min=0, max=4, description="Amplitude of wave 2", style=ws),
    fA=widgets.IntSlider(100, min=20, max=20000, description="Frequency of wave 1", style=ws),
    fB=widgets.IntSlider(10000, min=20, max=20000, description="Frequency of wave 2", style=ws),
    noise=widgets.FloatSlider(3, min=0, max=4, description="Amplitude of noise", style=ws),
    f0=widgets.IntSlider(0, 0, num_windows+1, description="First feature"),
    f1=widgets.IntSlider(1, 0, num_windows+1, description="Second feature"),
    K=widgets.IntSlider(5,1,10, description="Num neighbors"),
    weights=widgets.Select(options=['uniform', "distance"], value='distance', description='Weights:', disabled=False))
def generate_sound(A: float, B:float, fA:float, fB:float, noise: float, f0:int, f1:int, weights:str, K:int):
    T = 2
    pA, pB = 0, 0
    sample_rate = 44100
    _, signal = generate_noisy_signal(sample_rate, np.array([A, B]), np.array([fA, fB]), np.array([pA, pB]),
                                      noise=noise, T=T)
    audio = Audio(signal, rate=sample_rate, autoplay=True)
    display(audio)
    target_labels = list(target_map.keys())
    knn = KNeighborsClassifier(n_neighbors=K, weights=weights)
    knn.fit(features, labels)

    f = extract_features(signal, num_windows)
    predicted_label = knn.predict(f.reshape(1,-1))
    print(f"Predict as a {target_map[predicted_label[0]]} ({predicted_label[0]}) frequent sound")

    formatter = plt.FuncFormatter(lambda i, *args: target_map[i])
  
    c_f = extract_features(signal, num_windows=num_windows)
    plt.scatter(c_f[f0], c_f[f1], [50], marker="x", c=predicted_label[0], label="Test", zorder=2)
    plt.scatter(features[:,f0], features[:,f1], c=labels,
                cmap=plt.cm.get_cmap('viridis', len(target_labels)), label="Training data", zorder=1)
    plt.colorbar(ticks=target_labels, format=formatter)
    plt.legend()
    plt.xlabel(feature_names[f0])
    plt.ylabel(feature_names[f1])



interactive(children=(FloatSlider(value=0.5, description='Amplitude of wave 1', max=4.0, style=SliderStyle(des…